In [1]:
%load_ext autoreload
%autoreload now

In [2]:
from document_segmentation import settings
from document_segmentation.pagexml import GeneraleMissivenDataset

dataset = GeneraleMissivenDataset.from_csv(settings.GENERALE_MISSIVEN_SHEET)

ID in TANAP database                                                                       2047.0
Inv.nr. Nationaal Archief (1.04.02)                                                          1087
Deel v. inventarisnummer                                                                      NaN
Kamer                                                                                   Amsterdam
Beginfolio                                                                                  139.0
Eindfolio                                                                                   139.0
Beginscan                                                                                   299.0
Eindscan                                                                                    299.0
Enkele of dubbele pagina op scan                                                            Enkel
RGP Deel waarin de missive is opgenomen                                                       NaN
RGP pagina waarop de

In [3]:
dataset[5000]

(PageXMLScan(
 	id=NL-HaNA_1.04.02_1142_0204.jpg, 
 	type=['physical_structure_doc', 'text_region', 'pagexml_doc', 'scan'], 
 	stats={"lines": 84, "words": 1163, "text_regions": 13, "columns": 0, "extra": 0, "pages": 0}
 ),
 <Label.IN: 2>)

In [4]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:
training_dataset = dataset[:100]
test_dataset = dataset[100:200]

In [6]:
tagger.train_(training_dataset, epochs=50)

  2%|▏         | 1/50 [44:40<36:28:52, 2680.26s/epoch]

[Loss:	1.319]


  4%|▍         | 2/50 [44:40<14:43:01, 1103.78s/epoch]

[Loss:	1.213]


  6%|▌         | 3/50 [44:40<7:49:53, 599.87s/epoch]  

...

[Loss:	0.664]


 98%|█████████▊| 49/50 [44:50<00:00,  4.71epoch/s]

[Loss:	0.664]


100%|██████████| 50/50 [44:50<00:00, 53.81s/epoch]

[Loss:	0.664]


In [7]:
# Accuracy on training data

tagger.accuracy(test_dataset)

 25%|██▌       | 1/4 [01:06<03:20, 66.72s/batch]

[Batch Accuracy:	0.812]
[Total Accuracy:	0.812]


 50%|█████     | 2/4 [01:54<01:51, 55.74s/batch]

[Batch Accuracy:	0.750]
[Total Accuracy:	0.781]


 75%|███████▌  | 3/4 [03:04<01:02, 62.18s/batch]

[Batch Accuracy:	0.656]
[Total Accuracy:	0.740]


100%|██████████| 4/4 [03:15<00:00, 48.93s/batch]

[Batch Accuracy:	0.500]
[Total Accuracy:	0.730]


0.73

In [8]:
from document_segmentation.pagexml.dataset import Label


preds = tagger(test_dataset)
true_labels = test_dataset.labels()

print("\t".join(("True Label", "Predicted Label", "Correct?", "Predicted Scores")))
for true_label, pred, pred_label in zip(true_labels, preds, preds.argmax(dim=1)):
    print(
        "\t".join(
            (
                str(true_label),
                str(Label(pred_label.item() + 1)),
                str(Label(pred_label.item() + 1) == true_label),
                str(pred.tolist()),
            )
        )
    )

True Label	Predicted Label	Correct?	Predicted Scores
Label.IN	Label.BEGIN	False	[0.949296772480011, 0.015817411243915558, 0.03488577529788017]
Label.IN	Label.BEGIN	False	[0.916903018951416, 0.06631925702095032, 0.01677776873111725]
Label.IN	Label.BEGIN	False	[0.6955565214157104, 0.29165056347846985, 0.012792933732271194]
Label.IN	Label.IN	True	[0.20285220444202423, 0.7844436168670654, 0.012704157270491123]
Label.IN	Label.IN	True	[0.09488048404455185, 0.8809361457824707, 0.02418336272239685]
Label.IN	Label.IN	True	[0.39092180132865906, 0.5543187260627747, 0.054759442806243896]
Label.IN	Label.IN	True	[0.38991081714630127, 0.5638369917869568, 0.046252112835645676]
Label.IN	Label.BEGIN	False	[0.5097949504852295, 0.463417649269104, 0.02678738348186016]
Label.IN	Label.IN	True	[0.29404526948928833, 0.6849154829978943, 0.021039187908172607]
Label.IN	Label.IN	True	[0.3583122193813324, 0.6225782036781311, 0.01910967379808426]
Label.IN	Label.IN	True	[0.13568870723247528, 0.8365280032157898, 0.027

In [9]:
from torchview import draw_graph

model_graph = draw_graph(tagger)
print(model_graph.visual_graph)

RuntimeError: Only one of (input_data, input_size) should be specified.